In [1]:
import torch
import sys
import pickle
from transformers import DistilBertTokenizer
sys.path.append('../')
from distilbert_uncased_model import DistilBERTClass
CHECKPOINTLOC = '../final_model/Production_Model_v1'

c:\Users\csarc\anaconda3\envs\netskope\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

model = DistilBERTClass()
checkpoint = torch.load(CHECKPOINTLOC)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [3]:
print(model)


DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

In [4]:
with open('../Data/index_label_mapping.pkl','rb') as file:
    encoder = pickle.load(file)

encoder

{'Job Role': {0: 'DEVELOPMENT',
  1: 'GOVERNANCE RISK COMPLIANCE',
  2: 'INFORMATION SECURITY',
  3: 'IT GENERAL',
  4: 'NETWORKING',
  5: 'NON-ICP',
  6: 'SYSTEMS'},
 'Job Function': {0: 'ENGINEERING',
  1: 'IT',
  2: 'NON-ICP',
  3: 'PROCUREMENT',
  4: 'RISK/LEGAL/COMPLIANCE'},
 'Job Level': {0: 'C-LEVEL',
  1: 'CONTRIBUTOR',
  2: 'DIRECTOR',
  3: 'EXECUTIVE',
  4: 'MANAGER',
  5: 'UNKNOWN'}}

In [5]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)

In [6]:
with open('../Data/lead_data.pkl','rb') as file:
    lead_data = pickle.load(file)

lead_data.head()

,Title,Job Function,Job Role,Job Level
0,MANAGER-CYBERSECURITY,IT,INFORMATION SECURITY,MANAGER
1,"MANAGER, INFORMATION SECURITY",IT,INFORMATION SECURITY,MANAGER
2,USER EXPERIENCE ANALYST,ENGINEERING,DEVELOPMENT,CONTRIBUTOR
3,NETWORK SPECIALIST,IT,NETWORKING,CONTRIBUTOR
4,DIRECTOR OF PRIVACY AND COMPLIANCE,IT,INFORMATION SECURITY,DIRECTOR


In [11]:
lead_data['Sequence_Length_Tokens'] = lead_data['Title'].apply(lambda x: len(tokenizer.encode(x)))

In [12]:
lead_data.head()

,Title,Job Function,Job Role,Job Level,Sequence_Length_Tokens
0,MANAGER-CYBERSECURITY,IT,INFORMATION SECURITY,MANAGER,8
1,"MANAGER, INFORMATION SECURITY",IT,INFORMATION SECURITY,MANAGER,6
2,USER EXPERIENCE ANALYST,ENGINEERING,DEVELOPMENT,CONTRIBUTOR,5
3,NETWORK SPECIALIST,IT,NETWORKING,CONTRIBUTOR,4
4,DIRECTOR OF PRIVACY AND COMPLIANCE,IT,INFORMATION SECURITY,DIRECTOR,7


In [13]:
# How many data samples exceed 64 tokens?
(lead_data['Sequence_Length_Tokens'] > 64).sum()

0

In [14]:
# Length of maximum sequence?
lead_data['Sequence_Length_Tokens'].max()

46